In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import time


In [2]:
# Define the feedforward neural network
class SimpleANN(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, num_classes=10):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [3]:
# Hyperparameters
input_size = 784  # 28x28 images flattened
hidden_size = 128
num_classes = 10
num_epochs = 10
batch_size = 64
learning_rate = 0.001

In [4]:
# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████| 9912422/9912422 [00:34<00:00, 285993.73it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████| 28881/28881 [00:00<00:00, 179414.57it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████| 1648877/1648877 [00:07<00:00, 209671.48it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|███████████████████████████████████| 4542/4542 [00:00<00:00, 775136.46it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# Initialize the model, loss function, and optimizer
model = SimpleANN(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


def train():
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (images, labels) in enumerate(train_loader):
            # Flatten images from (batch_size, 1, 28, 28) to (batch_size, 784)
            images = images.view(-1, 28*28)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 28*28)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')


if __name__ == "__main__":
    import time

    # Measure training time
    start_time = time.time()
    train()
    end_time = time.time()

    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")

    # Measure inference time
    start_time = time.time()
    test()
    end_time = time.time()

    inference_time = end_time - start_time
    print(f"Inference time: {inference_time:.2f} seconds for the entire test set")

    per_sample_inference_time = inference_time / len(test_dataset)
    print(f"Inference time per sample: {per_sample_inference_time:.6f} seconds")



Epoch [1/10], Step [100/938], Loss: 0.3967
Epoch [1/10], Step [200/938], Loss: 0.4669
Epoch [1/10], Step [300/938], Loss: 0.5133
Epoch [1/10], Step [400/938], Loss: 0.3334
Epoch [1/10], Step [500/938], Loss: 0.3183
Epoch [1/10], Step [600/938], Loss: 0.4883
Epoch [1/10], Step [700/938], Loss: 0.3142
Epoch [1/10], Step [800/938], Loss: 0.3690
Epoch [1/10], Step [900/938], Loss: 0.0937
Epoch [2/10], Step [100/938], Loss: 0.0834
Epoch [2/10], Step [200/938], Loss: 0.2553
Epoch [2/10], Step [300/938], Loss: 0.1195
Epoch [2/10], Step [400/938], Loss: 0.1268
Epoch [2/10], Step [500/938], Loss: 0.3200
Epoch [2/10], Step [600/938], Loss: 0.2317
Epoch [2/10], Step [700/938], Loss: 0.1409
Epoch [2/10], Step [800/938], Loss: 0.1032
Epoch [2/10], Step [900/938], Loss: 0.1073
Epoch [3/10], Step [100/938], Loss: 0.1596
Epoch [3/10], Step [200/938], Loss: 0.2033
Epoch [3/10], Step [300/938], Loss: 0.0600
Epoch [3/10], Step [400/938], Loss: 0.2193
Epoch [3/10], Step [500/938], Loss: 0.1744
Epoch [3/10

In [6]:


# Save the model to a file
model_path = "simple_ann.pth"
torch.save(model.state_dict(), model_path)

# Measure the size of the model file
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Convert bytes to MB
print(f"Model size: {model_size:.2f} MB")

Model size: 0.39 MB
